## Estructura de los Datos

### Cash_Request (CR)



* **CR.id**:
  Unique ID of Cash Request.

* **CR.user_id**:
  Unique ID of the user who requested the cash advance.

* **CR.deleted_account_id**:
  If a user delete his account, we are replacing the user_id by this id. 
  
  It corresponds to a unique ID in the deleted account table with some keys information saved for fraud-fighting purposes (while respecting GDPR regulation).

* **CR.amount**:
  Amount of the Cash Request.

* **CR.created_at**:
  Timestamp of the CR creation.

* **CR.updated_at**:
  Timestamp of the latest CR's details update **(= update of at least one column in this table)**.

* **CR.reimbursement_date**:
  Planned reimbursement date. The user card will be charged at this date.

* **CR.cash_request_received_date**:
  Date of the receipt of the CR. Based on user's bank history.

* **CR.money_back_date**:
  Date where the CR was considered as money back. 
  
  It's either the paid_by_card date or the date were we considered that's the direc debit have low odds to be rejected (based on business rules) 

* **CR.send_at**:
  Timestamp of the funds's transfer.

---

* **CR.reco_creation**:
  Timestamp of the recovery creation.
* **CR.reco_last_update**:
  Timestamp of the last recovery case update. Can be used to determine the incident closure date.
* **CR.moderated_at**:
  Timestamp of the manual review. Only filled if the CR needed a manual review.

---
Columnas que figuran en la documentacion per que NO TENEMOS EN LA BASE DE DATOS:
---
* **(CR.reason)**:
  Filled only if the CR was manually reviewed and rejected. **That's the rejection's reason displayed in-app**. 

* **(CR.cash_request_debited_date)**:
  Filled only if a SEPA direct debit was sent. It's the date were the latest direct debit was seen on the user account.
---

* **CR.status**:     (23970 registros)
  Status of the CR. Possible values are: 

  | Campo                                                     | Registros | Descripcion                                                  |
  | --------------------------------------------------------- | --------: | ------------------------------------------------------------ |
  | **money_back**:                                           |     16397 | El CR fue reembolsado exitosamente.(The CR was successfully reimbursed.) |
  | **active**:                                               |        59 | Los fondos fueron recibidos en la cuenta del cliente. (Funds were received on the customer account.) |
  | **direct_debit_sent**:                                    |        34 | Se envió un débito directo SEPA, pero aún no se confirma el resultado. (We sent/scheduled a SEPA direct debit to charge the customer account. The result of this debit is not yet confirmed) |
  |                                                           |           |                                                              |
  | **rejected**:                                             |      6568 | El CR necesitó una revisión manual y fue rechazado. (The CR needed a manual review and was rejected) |
  | **direct_debit_rejected**:                                |       831 | El intento de débito directo SEPA falló. (Our last attempt of SEPA direct debit to charge the customer was rejected ) |
  | **transaction_declined**:                                 |        48 | No se pudo enviar el dinero al cliente. (We failed to send the funds to the customer) |
  | **canceled**:                                             |        33 | El usuario no confirmó el CR en la app, fue cancelado automáticamente. (The user didn't confirm the cash request in-app, we automatically canceled it) |
  | **En los datos proporcionados, NO aparecen los valores:** |           |                                                              |
  | **approved**:                                             |         0 | CR is a 'regular' one (= without fees) and was approved either automatically or manually. Funds will be sent aprox. 7 days after the creation. |
  | **money_sent**:                                           |         0 | We transferred the fund to the customer account. Will change to active once we detect that the user received the funds (using user's bank history). |
  | **pending**:                                              |         0 | The CR is pending a manual review from an analyst.           |
  | **waiting_user_confirmation**:                            |         0 | The user needs to confirm in-app that he want the CR (for legal reasons) |
  | **waiting_reimbursement**:                                |         0 | We were not able to estimate a date of reimbursement, the user needs to choose one in the app. |

     


* **CR.transfer_type**:
  Possible values are:

  | Campo        | Registros | Descripcion                                                  |
  | ------------ | --------: | ------------------------------------------------------------ |
  | **instant**: |     13882 | El usuario eligió recibir el adelanto instantáneamente. (user choose not received the advance instantly) |
  | **regular**: |     10086 | El usuario eligió no pagar inmediatamente y esperar la transferencia. (user choose to not pay and wait for the transfer) |

  


* **CR.recovery_status**:
  Possible values are:

  | Campo                     | Registros | Descripcion                                                  |
  | ------------------------- | --------: | ------------------------------------------------------------ |
  | **NULL (Nice)**:          |     20639 | El CR nunca tuvo un incidente de pago. (Null if the cash request never had a payment incident.) |
  | **completed**:            |      2467 | El incidente de pago fue resuelto (el CR fue reembolsado). (the payment incident was resolved **(=the cash request was reimbursed)**) |
  | **pending**:              |       845 | El incidente de pago aún está abierto. (the payment incident still open) |
  | **pending_direct_debit**: |        16 | El incidente de pago sigue abierto, pero se ha lanzado un débito directo SEPA. (the payment incident still open but a SEPA direct debit is launched) |
  | **cancelled**:            |         1 | ?? No figura a la documentacio                               |


### Fees (FE)

* **FE.id**:
  Unique ID of the fee object.

* **FE.cash_request_id**:
  Unique ID of the CR linked to this fee.

* **FE.total_amount**:
  Amount of the fee (including VAT). **TODO: Es un importe o un porcentaje ??**

* **FE.reason**:
  Description of the fee.

* **FE.created_at**:
  Timestamp of the fee's creation.

* **FE.updated_at**:
  Timestamp of the latest fee's details update.

* **FE.paid_at**:
  Timestamp of the fee's payment.

* **FE.from_date**:
  Apply only to postpone fees. Initial date of reimbursement for the CR.

* **FE.to_date**:
  Apply only to postpone fees. New date of reimbursement for the CR.


* **FE.type**:
  Type of fee. Possible values are:

  | Campo                | Registros | Descripcion                                                  |
  | -------------------- | --------: | ------------------------------------------------------------ |
  | **instant_payment**: |     11095 | Fees por adelanto instantáneo. (fees for instant cash request (send directly after user's request, **through SEPA Instant Payment**) ) |
  | **postpone**:        |      7766 | Fees por la solicitud de posponer un reembolso. (fees created when a user want to postpone the reimbursment of a CR) |
  | **incident**:        |      2196 | Fees por fallos de reembolsos. (fees for failed reimbursement. Created after a failed direct debit) |
  | **split_payment**:   |         0 | Fees por pago fraccionado (en caso de un incidente).<br/>(futures fees for split payment (in case of an incident, we'll soon offer the possibility to our users to reimburse in multiples installements)) |


* **FE.status**:
  (= does the fees was successfully charged). Possibles values are:

  | Campo          | Registros | Descripcion                                                  |
  | -------------- | --------: | ------------------------------------------------------------ |
  | **accepted**:  |     14841 | El fee fue cobrado exitosamente. (fees were successfully charged) |
  | **cancelled**: |      4934 | El fee fue creado pero cancelado por algún motivo. (fee was created and cancelled for some reasons. <br/>It's used to fix issues with fees but it mainly concern postpone fees who failed. <br/>We are charging the fees at the moment of the postpone request. <br/>If it failed, the postpone is not accepted and the reimbursement date still the same.) |
  | **rejected**:  |      1194 | El último intento de cobrar el fee falló. (the last attempt to charge the fee failed.) |
  | **confirmed**: |        88 | El usuario completó una acción que creó un fee. (the user made an action who created a fee. <br/>It will normally get charged at the moment of the CR's reimbursement. In some rare cases, postpones are confirmed without being charges due to a commercial offer.) |


* **FE.category**:
  Describe the reason of the incident fee. 2 possibles values:

  | Campo                       | Registros | Descripcion                                                  |
  | --------------------------- | --------: | ------------------------------------------------------------ |
  | **rejected_direct_debit**:  |     18861 | Fees creados cuando el banco del usuario rechaza el primer débito directo. (fees created when user's bank rejects the first direct debit) |
  | **month_delay_on_payment**: |      1599 | Fees creados cada mes hasta que el incidente se cierre. (fees created every month until the incident is closed) |
  | **NULL**:                   |       597 | ?? No figura a la documentacio                               |


* **FE.charge_moment**:
  When the fee will be charge. 2 possibles values:

  | Campo       | Registros | Descripcion                                                  |
  | ----------- | --------: | ------------------------------------------------------------ |
  | **after**:  |     16720 | El fee se cobra cuando el CR es reembolsado. (the fee should be charged at the moment of the CR's reimbursement) |
  | **before**: |      4337 | El fee se cobra en el momento de su creación. (the fee should be charged at the moment of its creation) |

## EDA

In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm

cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
df_jo = pm.df('df_jo')

#df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = pm.sort("df_jo", ['created_at','created_at_fe']).reset_index()
df_jo = df_jo.drop(columns=['index'])

#df_jo = df_jo.drop(columns=['Mes_created_at'])
df_jo_cp = df_jo.copy()
df_jo_cp['cr_received_date'] = df_jo_cp.cash_request_received_date
#df_jo.info()

In [9]:
counts = cr_cp.status.value_counts()
display(counts)

counts = cr_cp.transfer_type.value_counts()
display(counts)

counts = cr_cp.recovery_status.value_counts()
display(counts)

status
money_back               16395
rejected                  6568
direct_debit_rejected      831
active                      59
transaction_declined        48
direct_debit_sent           34
canceled                    33
Name: count, dtype: int64

transfer_type
instant    13882
regular    10086
Name: count, dtype: int64

recovery_status
nice                    20639
completed                2467
pending                   845
pending_direct_debit       16
cancelled                   1
Name: count, dtype: int64

In [12]:
counts = fe_cp.status.value_counts()
display(counts)

counts = fe_cp.type.value_counts()
display(counts)

counts = fe_cp.category.value_counts()
display(counts)

counts = fe_cp.charge_moment.value_counts()
display(counts)


status
accepted     14841
cancelled     4934
rejected      1194
confirmed       88
Name: count, dtype: int64

type
instant_payment    11095
postpone            7766
incident            2196
Name: count, dtype: int64

category
nice                      18861
rejected_direct_debit      1599
month_delay_on_payment      597
Name: count, dtype: int64

charge_moment
after     16720
before     4337
Name: count, dtype: int64

In [34]:
df = df_jo.copy()
timeFormat ='S' #D
df['created_at'] = df['created_at'].dt.to_period(timeFormat) #'Min')
df['created_at_fe'] = df['created_at_fe'].dt.to_period(timeFormat) #'Min')
df['updated_at'] = df['updated_at'].dt.to_period(timeFormat) #'Min')

df['to_receive_ini'] = pd.to_timedelta(df['to_receive_ini']).round(timeFormat)
#df['to_receive_ini'] = df['to_receive_ini'].timedelta(seconds=math.ceil(df['to_receive_ini'].total_seconds()))

df['to_receive_bank'] = pd.to_timedelta(df['to_receive_bank']).round(timeFormat)
df['to_reimbur'] = pd.to_timedelta(df['to_reimbur']).round(timeFormat)
df['to_reimbur_cash'] = pd.to_timedelta(df['to_reimbur_cash']).round(timeFormat)
df['to_end'] = pd.to_timedelta(df['to_end']).round(timeFormat)
df['to_send'] = pd.to_timedelta(df['to_send']).round(timeFormat)

df['money_back_date'] = df['money_back_date'].dt.to_period(timeFormat)
df['send_at'] = df['send_at'].dt.to_period(timeFormat)
df['paid_at'] = df['paid_at'].dt.to_period(timeFormat)
df['moderated_at'] = df['moderated_at'].dt.to_period(timeFormat)
df['from_date'] = df['from_date'].dt.to_period(timeFormat)
df['to_date'] = df['to_date'].dt.to_period(timeFormat)

df['reco_creation'] = df['reco_creation'].dt.to_period(timeFormat)
df['reco_last_update'] = df['reco_last_update'].dt.to_period(timeFormat)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
display(df.head(5))

,id_cr,id_fe,fe_cr_id,user_id,active,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status,updated_at,reco_creation,reco_last_update,updated_at_fe,Mes_created_at,cash_request_received_date,n_fees,n_backs,created_at_slot,created_at_slot_h,created_at_dow
0,3,0,NaN,47,1,2019-11-19 13:57:53,NaT,1.0,NaN,canceled,NaN,regular,nice,NaT,NaT,16 days 09:02:06,NaT,NaT,NaT,NaT,NaT,NaT,2019-12-05 23:00:00.000000,NaT,NaN,2019-11-20 12:16:50,NaN,NaN,NaT,NaT,nice,2020-12-14 10:25:57,NaT,NaT,NaT,2019-11,NaT,0,0,7,13-Mañana,1
1,4,0,NaN,99001309,0,2019-12-09 14:47:35,NaT,100.0,NaN,money_back,NaN,regular,nice,NaT,NaT,7 days 08:12:25,NaT,0 days,NaT,NaT,NaT,2019-12-16 23:00:00,2019-12-16 23:00:00.000000,NaT,NaN,2019-12-09 14:58:28,NaN,NaN,NaT,NaT,nice,2020-11-04 12:54:52,NaT,NaT,NaT,2019-12,NaT,0,1,14,14-Tarde,0
2,5,0,NaN,804,1,2019-12-10 19:05:21,NaT,100.0,NaN,rejected,NaN,regular,nice,NaT,NaT,30 days 00:00:00,NaT,NaT,NaT,NaT,NaT,NaT,2020-01-09 19:05:21.596363,NaT,NaN,2019-12-11 16:47:42,NaN,NaN,NaT,NaT,nice,2019-12-11 16:47:42,NaT,NaT,NaT,2019-12,NaT,0,0,14,19-Tarde,1
3,6,0,NaN,812,1,2019-12-10 19:05:48,NaT,100.0,NaN,direct_debit_rejected,NaN,regular,nice,0 days 04:54:11,NaT,57 days 03:54:11,NaT,NaT,NaT,NaT,2019-12-11,NaT,2020-02-05 23:00:00.000000,NaT,NaN,2019-12-11 09:40:33,NaN,NaN,NaT,NaT,pending,2020-12-18 22:47:47,2020-03-20 14:12:53,2020-12-18 22:47:48,NaT,2019-12,2019-12-11,0,0,14,19-Tarde,1
4,7,0,NaN,191,1,2019-12-10 19:13:35,NaT,100.0,NaN,rejected,NaN,regular,nice,NaT,NaT,30 days 00:00:00,NaT,NaT,NaT,NaT,NaT,NaT,2020-01-09 19:13:35.825041,NaT,NaN,2019-12-11 09:46:59,NaN,NaN,NaT,NaT,nice,2019-12-11 09:46:59,NaT,NaT,NaT,2019-12,NaT,0,0,14,19-Tarde,1
